### DubvserseAI 


In [ ]:
! pip install transformers datasets evaluate sacrebleu

In [2]:
from huggingface_hub import notebook_login

notebook_login()

### Dataset

dataset : https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [5]:
from datasets import load_dataset , load_metric
from datasets import Dataset

# Load the dataset
raw_dataset = load_dataset("cfilt/iitb-english-hindi",split='train[:20000]')
metric = load_metric("sacrebleu")

<ipython-input-5-678cda7c59c3>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [6]:
raw_datasets = raw_dataset.train_test_split(test_size=0.2)

In [7]:
raw_datasets["train"][0]

{'translation': {'en': 'Confim the following information:',
  'hi': 'सूचना दिखाने वाली लकीर'}}

In [8]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'A generic Xlib project', 'hi': 'जेनेरिक Xlib प्रोजेक्ट'}"
1,"{'en': 'Common additional Java compiler flags for all targets in this group.', 'hi': 'सामान्य अतिरिक्त जावा फ्लैग्स के लिए सभी लक्ष्य इंच यह समूह. '}"
2,"{'en': 'Username:', 'hi': 'उपयोक्ता नामः'}"
3,"{'en': 'Directory filter:', 'hi': 'डायरेक्ट्री'}"
4,"{'en': 'Show marks', 'hi': 'दिखाएँ'}"


In [10]:
from transformers import AutoTokenizer
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
!pip install sacremoses

In [12]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "hi_IN"

In [13]:
print(tokenizer("Hello, this one sentence!"))
print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [12110, 2, 90, 131, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [[12110, 2, 90, 131, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [14]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Hindi: "
else:
    prefix = ""

In [15]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
preprocess_function(raw_datasets['train'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[55822, 363, 4, 960, 759, 20, 0], [4417, 20, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1]], 'labels': [[753, 1688, 1879, 6677, 0], [17503, 0]]}

In [17]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [18]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [19]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [28]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"DubverseAI",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/SRDdev/DubverseAI into local empty directory.


In [29]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.150500,0.205304,71.610800,5.841800
2,0.105700,0.188720,72.346800,5.995300


Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


TrainOutput(global_step=2000, training_loss=0.1282172145843506, metrics={'train_runtime': 1498.4139, 'train_samples_per_second': 21.356, 'train_steps_per_second': 1.335, 'total_flos': 139635839729664.0, 'train_loss': 0.1282172145843506, 'epoch': 2.0})

In [30]:
trainer.push_to_hub()

To https://huggingface.co/SRDdev/DubverseAI
   0a344c1..6ae573a  main -> main

   0a344c1..6ae573a  main -> main

To https://huggingface.co/SRDdev/DubverseAI
   6ae573a..5bc24f7  main -> main

   6ae573a..5bc24f7  main -> main



'https://huggingface.co/SRDdev/DubverseAI/commit/6ae573aca1d3322264e2e56479afdc543a61282b'

### Prompts / Inference

In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("SRDdev/DubverseAI")

model = AutoModelForSeq2SeqLM.from_pretrained("SRDdev/DubverseAI")

In [40]:
from transformers import pipeline

translate = pipeline('translation', tokenizer=tokenizer,model=model)
translate("My name is Shreyas")[0]['translation_text']

'मेरा नाम है लीयायास'

In [42]:
# English sentences for Hindi translation
sentences = [
    "Hello, how are you?",
    "I love you.",
    "What is your name?",
    "Can you help me, please?",
    "I am going to the market.",
    "Happy birthday!",
    "Where is the nearest hospital?",
    "I don't understand.",
    "How much does it cost?",
    "Please wait for a moment."
]

# Iterate over the sentences and perform translation
for sentence in sentences:
    print(translate(f"{sentence}")[0]['translation_text'])
    print("---------------------------")

हैलो, आप कैसे हैं?
---------------------------
मैं तुमसे प्यार करता हूँ.
---------------------------
आपका GroupWise आईडी क्या है?
---------------------------
कृपया, मेरी मदद कर सकते हैं?
---------------------------
मैं बाजार के लिए जा रहा हूँ.
---------------------------
जन्मदिन मुबारक हो!
---------------------------
कहां सबसे निकट अस्पताल है?
---------------------------
मैं नहीं समझा.
---------------------------
इसकी कीमत कितनी है?
---------------------------
कृपया एक पल के लिए इंतजार करें.
---------------------------
